## Cálculo da emissão e distância percorrida 

Nesse notebook é calculada a distância percorrida entre os pontos e a emissão de poluentes

In [1]:
import pandas as pd
import random
from utils.save_csv import save_csv
from utils.load_csv import load_csv
from datetime import datetime

In [2]:
consumo_diesel = load_csv('consumo_diesel.csv')

veiculos = load_csv('veiculos.csv')

posicoes_df = load_csv('posicoes_onibus.csv')

In [3]:
consumo_diesel['emissa_no2_l_diesel'] = [random.uniform(0.02, 0.05) for _ in range(len(consumo_diesel))]
consumo_diesel['emissa_co2_l_diesel'] = [random.uniform(0.05, 0.1) for _ in range(len(consumo_diesel))]

In [4]:
consumo_diesel.head()

,tecnologia,sem_ar_l_km,sem_ar_kg_km,com_ar_l_km,com_ar_kg_km,emissa_no2_l_diesel,emissa_co2_l_diesel
0,Miniônibus,0.30,0.252,0.35,0.294,0.043020,0.050118
1,Midiônibus,0.40,0.336,0.47,0.395,0.043683,0.089961
2,Básico,0.46,0.386,0.53,0.445,0.047989,0.059462
3,Padron,0.55,0.462,0.63,0.529,0.027724,0.084527
4,Articulado (18m),0.71,0.596,0.80,0.672,0.040437,0.076177


In [5]:
def get_tipo_onibus(id_onibus):

    onibus = veiculos[veiculos['id_veiculo']==id_onibus]

    return onibus['modelo'].unique()[0]

In [6]:
def get_fator_consumo(tipo_veiculo):

    cosumo_modelo = consumo_diesel[consumo_diesel['tecnologia']==tipo_veiculo]

    return cosumo_modelo['com_ar_l_km'].unique()[0]

def get_consumo_co2(tipo_veiculo, km_rodados):

    consumo_l = get_fator_consumo(tipo_veiculo)
    consumo_fato = consumo_l * km_rodados

    fator_emissao_modelo = consumo_diesel[consumo_diesel['tecnologia']==tipo_veiculo]
    fator_emissao_co2_modelo = fator_emissao_modelo['emissa_co2_l_diesel'].unique()[0]
    emissao_co2 = fator_emissao_co2_modelo * consumo_fato

    return emissao_co2

def get_consumo_no2(tipo_veiculo, km_rodados):

    consumo_l = get_fator_consumo(tipo_veiculo)
    consumo_fato = consumo_l * km_rodados

    fator_emissao_modelo = consumo_diesel[consumo_diesel['tecnologia']==tipo_veiculo]
    fator_emissao_no2_modelo = fator_emissao_modelo['emissa_no2_l_diesel'].unique()[0]
    emissao_no2 = fator_emissao_no2_modelo * consumo_fato

    return emissao_no2

In [7]:
data_frame = []

for id_onibus in posicoes_df['id_onibus'].unique():
    df_onibus = posicoes_df[posicoes_df['id_onibus'] == id_onibus].reset_index(drop=True)
    df_onibus.sort_values(by='t', inplace=True)
    for i, row in df_onibus.iterrows():
 
        if i < len(df_onibus)-1:
            x_atual = row['x']
            x_depois = df_onibus.loc[i+1, 'x']
            y_atual = row['y']
            y_depois = df_onibus.loc[i+1, 'y']
            momento_inicial = row['t']
            momento_final = df_onibus.loc[i+1, 't']
            tipo_veiculo = get_tipo_onibus(row['id_onibus'])
            litro_diesel = get_fator_consumo(tipo_veiculo)
            
            #mas tem que reprojetar para SIRGAS 2000
            dist = (x_depois - x_atual)**2 + (y_depois - y_atual)**2
            
            dados = {
                'id_onibus' : row['id_onibus'],
                'distancia_percorrida' : dist,
                'ponto_inicial' : [x_atual, y_atual],
                'ponto_final' : [x_depois, y_depois],
                'momento_inicial' : momento_inicial,
                'momento_final' : momento_final,
                'modelo' : tipo_veiculo,
                'litro_diesel_km' : litro_diesel,
                'emissao_no2' : get_consumo_no2(tipo_veiculo, dist),
                'emissao_co2' : get_consumo_co2(tipo_veiculo, dist)
            }

            data_frame.append(dados)

df_final = pd.DataFrame(data_frame)

In [8]:
df_final = df_final.merge(
    veiculos[['id_veiculo', 'eletrico']],
    left_on='id_onibus',
    right_on='id_veiculo',
    how='left').drop(columns='id_veiculo')

In [9]:
dados_trips_layer = []

for onibus in df_final['id_onibus'].unique():

    df_onibus = df_final[df_final['id_onibus'] == onibus].reset_index(drop=True)
    coordinates = []
    timestamps = []
    emissao_total = 0
    df_onibus.sort_values(by='momento_inicial', inplace=True)
    for i, row in df_onibus.iterrows():
        coordinates.append(row['ponto_inicial'])
        stamp = datetime.fromisoformat(row['momento_inicial']).timestamp()
        timestamps.append(stamp)
        emissao_total += row['emissao_co2']

    dados_trips_layer.append({
        'id_onibus': onibus,
        'emissao_co2': emissao_total,
        'eletrico': row['eletrico'],
        'coordinates' : coordinates,
        'timestamps' : timestamps
    })


df_trips = pd.DataFrame(dados_trips_layer)


In [10]:
def init_on_zero(timestamp_list)->list[float]:

    inicial = timestamp_list[0]

    new_timestamps = [ts - inicial for ts in timestamp_list]

    return [int(ts) for ts in new_timestamps]


df_trips['timestamps'] = df_trips['timestamps'].apply(init_on_zero)

In [11]:
df_trips.head()

,id_onibus,emissao_co2,eletrico,coordinates,timestamps
0,1001,0.000005,True,"[[-46.61931627478385, -23.52014606122728], [-4...","[0, 60, 120, 180, 240, 300, 360, 420, 480, 540..."
1,1002,0.000017,True,"[[-46.57068238406227, -23.526040251946096], [-...","[0, 60, 120, 180, 240, 300, 360, 420, 480, 540..."
2,1003,0.000013,False,"[[-46.60140067568386, -23.522929524136508], [-...","[0, 60, 120, 180, 240, 300, 360, 420, 480, 540..."
3,1004,0.000027,True,"[[-46.56033996554972, -23.536729239415283], [-...","[0, 60, 120, 180, 240, 300, 360, 420, 480, 540..."
4,1005,0.000005,True,"[[-46.629765185206, -23.51957773218024], [-46....","[0, 60, 120, 180, 240, 300, 360, 420, 480, 540..."


In [12]:
coord_t = str(df_trips.loc[0, 'coordinates'])

In [13]:
eval(coord_t)

[[-46.61931627478385, -23.52014606122728],
 [-46.617316274783846, -23.51814606122728],
 [-46.61531627478384, -23.51614606122728],
 [-46.61331627478384, -23.514146061227283],
 [-46.61131627478384, -23.512146061227284],
 [-46.60931627478384, -23.510146061227285],
 [-46.60731627478384, -23.508146061227286],
 [-46.60531627478383, -23.506146061227287],
 [-46.60331627478383, -23.50414606122729],
 [-46.60131627478383, -23.50214606122729],
 [-46.599316274783824, -23.50014606122729],
 [-46.59731627478382, -23.49814606122729],
 [-46.59531627478382, -23.496146061227293],
 [-46.59331627478382, -23.49414606122729],
 [-46.59131627478381, -23.49214606122729],
 [-46.58931627478381, -23.490146061227296],
 [-46.58731627478381, -23.488146061227297],
 [-46.58531627478381, -23.4861460612273],
 [-46.583316274783805, -23.4841460612273],
 [-46.5813162747838, -23.4821460612273],
 [-46.5793162747838, -23.480146061227305],
 [-46.5773162747838, -23.478146061227303],
 [-46.5753162747838, -23.476146061227304],
 [-4

In [14]:
df_trips.loc[0, 'timestamps']

[0,
 60,
 120,
 180,
 240,
 300,
 360,
 420,
 480,
 540,
 600,
 660,
 720,
 780,
 840,
 900,
 960,
 1020,
 1080,
 1140,
 1200,
 1260,
 1320,
 1380,
 1440,
 1500,
 1560,
 1620,
 1680,
 1740,
 1800,
 1860,
 1920,
 1980,
 2040,
 2100,
 2160,
 2220,
 2280]

In [15]:
df_trips.to_csv('trips_layer.csv', index=False)

In [16]:
import pydeck as pdk

In [17]:
df_trips = df_trips[['coordinates', 'timestamps']]

In [18]:
df_trips = df_trips.head()

In [19]:
import pydeck as pdk

trips_layer = pdk.Layer(
    "TripsLayer",
    data=df_trips,
    get_path="coordinates",       # coluna com listas [lon, lat]
    get_timestamps="timestamps",  # coluna com timestamps
    get_color=[255, 0, 0],        # ou use uma cor baseada em 'eletrico' se quiser
    opacity=0.8,
    width_min_pixels=5,
    rounded=True,
    trail_length=240,             # ajusta conforme duração da viagem
    current_time=0                # tempo inicial
)

view_state = pdk.ViewState(
    latitude=-23.55,
    longitude=-46.57,
    zoom=11,
    pitch=45
)

r = pdk.Deck(layers=[trips_layer], initial_view_state=view_state)
r.to_html("trips.html")